# Course 4 : Adaptive step size for Euler Method 

In [ ]:
import numpy as np
from scipy.optimize import root
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

## Error estimator

We can construct an estimator of the local truncation error from the iterates of the explicit Euler method.
The local truncation error can be estimated from the second derivative of the solution :

$$ \mathcal{E}_n = \frac{\Delta t_n^2}{2} y''(t_n) + O(\Delta t_n^3).$$

When using an Euler-Cauchy or explicit Euler type solver, an estimator of the local truncation error can be given by :

$$
\frac{\Delta t_n}{2}  \left ( f(t_{n}, y_{n}) - f(t_{n-1}, y_{n-1})\right ), \qquad y''(t_n) \simeq \frac{f(t_{n}, y_{n}) - f(t_{n-1}, y_{n-1})}{\Delta t_n}.
$$

If we want to adapt the time step in order to obtain a local truncation error increased by a fixed tolerance $\epsilon$, we can choose the time step at time $n$ by the formula : 

$$
\Delta t_{n} =   \sqrt{\frac{2\epsilon \Delta t_{n}}{f(t_{n}, y_n) - f(t_{n-1}, y_{n-1})} },
$$

as long as an initial time step is also provided. 

In [ ]:
#####################################################
class ode_result:
    def __init__(self, y, t):
        self.y = y
        self.t = t

#####################################################
def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt))
    y[:,0] = yini

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*np.array(fcn(tn, yn))

    return ode_result(y, t)

#####################################################
def forward_euler_adapted(tini, tend, dtini, yini, fcn, tol=1e-6):

    t = [tini];

    yini = np.atleast_1d(yini)
    y = [yini]
        
    dt = dtini
    tn = tini
    it = 0
    
    while (tn<tend):
        yn = y[it]
        y.append(yn + dt*np.array(fcn(tn, yn)))
        tn += dt
        t.append(tn)
        ddy = abs(y[it+1]-yn)/dt
        dt = np.sqrt(2*tol/np.linalg.norm(ddy))
        if (tn+dt)>tend: dt = tend-tn
        it += 1
        
    y = np.stack(y, axis=1)
    t = np.array(t)
    
    return ode_result(y, t)

## Curtiss et Hirschfelder problem

We consider the following problem :

$$
\left\{ 
\begin{aligned} 
{\mathrm d}_t u(t) & = k \, \big(cos(t) - u(t)) \big) \quad \text{with } k > 1\\ 
u(0)  & = u_0 
\end{aligned} 
\right. 
$$

In [ ]:
class curtiss_model:

    def __init__(self, k):
        self.k = k

    def fcn(self, t, u) :
        k = self.k
        u_dot = k * (np.cos(t) - u)
        return u_dot

    def sol(self, uini, t0, t):
        k = self.k

        c0 = (uini - (k/(k*k + 1)) * (k*np.cos(t0) + np.sin(t0))) * np.exp(k*t0)
        u = (k/(k*k + 1)) * (k*np.cos(t) + np.sin(t)) +  c0 * np.exp(-k*t)
        return u

### Forward Euler

In [ ]:
uini = (2.,)
tini = 0.
tend = 0.5
k = 100.

cm = curtiss_model(k)
fcn = cm.fcn

fig =  make_subplots(rows=2, cols=1, vertical_spacing=0.1, subplot_titles=("Solution", "Error"))

nt = 400
sol = forward_euler(tini, tend, nt, uini, fcn)
err = np.abs(sol.y[0] - cm.sol(uini, tini, sol.t))

fig.add_trace(go.Scatter(x=sol.t, y=sol.y[0], name=f'fixed dt, nt={sol.t.size}', 
                         line_color='rgb(76,114,176)', mode='markers+lines', legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=err, name=f'fixed dt, nt={sol.t.size}', 
                         line_color='rgb(76,114,176)', mode='markers+lines', legendgroup='2'), row=2, col=1)


tol = np.array([1e-4, 1e-5, 1e-6])
for tol_i in tol:
    dtini = 1e-4
    sol_adapt = forward_euler_adapted(tini, tend, dtini, uini, fcn, tol=tol_i)
    err_adapt = np.abs(sol_adapt.y[0] - cm.sol(uini, tini, sol_adapt.t))

    fig.add_trace(go.Scatter(visible=False, x=sol_adapt.t, y=sol_adapt.y[0], name=f'adapted dt, nt={sol_adapt.t.size}', 
                             line_color='rgb(221,132,82)', mode='markers+lines', legendgroup='1'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_adapt.t, y=err_adapt, name=f'adapted dt, nt={sol_adapt.t.size}', 
                             line_color='rgb(221,132,82)', mode='markers+lines', legendgroup='2'), row=2, col=1)

fig.data[-2].visible = True
fig.data[-1].visible = True

# Create and add slider
steps = []
for i, tol_i in enumerate(tol):
    args = [{"visible": [(el==0) or (el==1) or (el==2*i+2) or (el==2*i+3) for el in range(len(fig.data))]}]
    step = dict(method="update", label = f"{tol_i:.2e}", args=args)
    steps.append(step)
sliders = [dict(active=2, currentvalue={'prefix': 'Tolerance = '}, steps=steps)]
    
legend=dict(x=0.70,y=0.98, bgcolor='rgba(0,0,0,0)')
fig.update_layout(sliders=sliders, height=800, legend=legend, legend_tracegroupgap=300, legend_groupclick="toggleitem")

### Backward Euler

In [ ]:
#####################################################
def backward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt))
    y[:,0] = yini

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*np.array(fcn(tip1, uip1))

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*np.array(fcn(tn, yn))
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        sol = root(g, y0, (yn, tn+dt))
        y[:,it+1] = sol.x

    return ode_result(y, t)

#####################################################
def backward_euler_adapted(tini, tend, dtini, yini, fcn, tol=1e-6):
    
    t = [tini];

    yini = np.atleast_1d(yini)
    y = [yini]
    
    dt = dtini
    tn = tini
    it = 0

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*np.array(fcn(tip1, uip1))
    
    
    while (tn<tend):
        yn = y[it]
        y0 = yn + dt*np.array(fcn(tn, yn))
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        sol = root(g, y0, (yn, tn+dt))
        y.append(sol.x)
        tn += dt
        t.append(tn)
        ddy = abs(y[it+1]-yn)/dt
        #dt = np.sqrt(2*tol/max(np.linalg.norm(ddy),1e-4))
        dt = np.sqrt(2*tol/np.linalg.norm(ddy))
        if (tn+dt)>tend: dt = tend-tn
        it += 1
        
    y = np.stack(y, axis=1)
    t = np.array(t)

    return ode_result(y, t)

In [ ]:
uini = (2.,)
tini = 0.
tend = 0.5
k = 100.

cm = curtiss_model(k)
fcn = cm.fcn

fig =  make_subplots(rows=2, cols=1, vertical_spacing=0.1, subplot_titles=("Solution", "Error"))

nt = 400
sol = backward_euler(tini, tend, nt, uini, fcn)
err = np.abs(sol.y[0] - cm.sol(uini, tini, sol.t))

fig.add_trace(go.Scatter(x=sol.t, y=sol.y[0], name=f'fixed dt, nt={sol.t.size}', 
                         line_color='rgb(76,114,176)', mode='markers+lines', legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=sol.t, y=err, name=f'fixed dt, nt={sol.t.size}', 
                         line_color='rgb(76,114,176)', mode='markers+lines', legendgroup='2'), row=2, col=1)


tol = np.array([1e-3, 1e-4, 1e-5, 1e-6])
for tol_i in tol:
    dtini = 1e-4
    sol_adapt = backward_euler_adapted(tini, tend, dtini, uini, fcn, tol=tol_i)
    err_adapt = np.abs(sol_adapt.y[0] - cm.sol(uini, tini, sol_adapt.t))

    fig.add_trace(go.Scatter(visible=False, x=sol_adapt.t, y=sol_adapt.y[0], name=f'adapted dt, nt={sol_adapt.t.size}', 
                             line_color='rgb(221,132,82)', mode='markers+lines', legendgroup='1'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_adapt.t, y=err_adapt, name=f'adapted dt, nt={sol_adapt.t.size}', 
                             line_color='rgb(221,132,82)', mode='markers+lines', legendgroup='2'), row=2, col=1)

fig.data[-2].visible = True
fig.data[-1].visible = True

# Create and add slider
steps = []
for i, tol_i in enumerate(tol):
    args = [{"visible": [(el==0) or (el==1) or (el==2*i+2) or (el==2*i+3) for el in range(len(fig.data))]}]
    step = dict(method="update", label = f"{tol_i:.2e}", args=args)
    steps.append(step)
sliders = [dict(active=3, currentvalue={'prefix': 'Tolerance = '}, steps=steps)]
    
legend=dict(x=0.70,y=0.98, bgcolor='rgba(0,0,0,0)')
fig.update_layout(sliders=sliders, height=800, legend=legend, legend_tracegroupgap=300, legend_groupclick="toggleitem")

## Explosion model

$$
d_{\tau} \widetilde{\theta} = f(\widetilde{\theta}) = \exp\Bigg(\frac{\widetilde{\theta}}{1 + \widetilde{\theta} \, / \, \beta}\Bigg) \Bigg(1 - \frac{\widetilde{\theta}}{\overline T_r} \Bigg)
$$

In [ ]:
class explosion_with_consumption_1eq_model:

    def __init__(self, Tr, beta):
        self.Tr = Tr
        self.beta = beta 

    def fcn(self, t, theta):
        beta = self.beta
        Tr = self.Tr
        theta_dot = np.exp(theta/(1+(theta/beta))) * (1-theta/Tr) 
        return theta_dot
    
    def jac(self, t, theta):
        beta = self.beta
        Tr = self.Tr
        exp = np.exp(theta/(1+(theta/beta)))
        return (exp/(1+theta/beta)**2) * (1-theta/Tr) - exp/Tr

### Euler explicit

In [ ]:
Tr = 200.0
beta = 10. 

emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
fcn = emwc1eq.fcn
jac = emwc1eq.jac

tini = 0.0
tend = 5.0

yini = (0.,)

fig =  make_subplots(rows=2, cols=1, vertical_spacing=0.1, subplot_titles=("Solution", "Error"))

sol_ref = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)

nt = 10001
sol = forward_euler(tini, tend, nt, yini, fcn)
sol_tmp = solve_ivp(fcn, (tini, tend), yini, method='Radau', t_eval=sol.t, atol=1.e-8, rtol=1.e-8)
err = np.abs(sol.y[0] - sol_tmp.y[0])

fig.add_trace(go.Scattergl(x=sol_ref.t, y=sol_ref.y[0], name=f'ref. sol.', 
                           line_color='rgb(85,168,104)', mode='markers+lines', legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scattergl(x=sol.t, y=sol.y[0], name=f'fixed dt, nt={sol.t.size}', 
                         line_color='rgb(76,114,176)', mode='markers+lines', legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scattergl(x=sol.t, y=err, name=f'fixed dt, nt={sol.t.size}', 
                         line_color='rgb(76,114,176)', mode='markers+lines', legendgroup='2'), row=2, col=1)


tol = np.array([1e-4, 1e-5, 1e-6])
for tol_i in tol:
    dtini = 1e-4
    sol_adapt = forward_euler_adapted(tini, tend, dtini, yini, fcn, tol=tol_i)
    sol_tmp = solve_ivp(fcn, (tini, tend), yini, method='Radau', t_eval=sol_adapt.t, atol=1.e-8, rtol=1.e-8)
    err_adapt = err = np.abs(sol_adapt.y[0] - sol_tmp.y[0])

    fig.add_trace(go.Scattergl(visible=False, x=sol_adapt.t, y=sol_adapt.y[0], name=f'adapted dt, nt={sol_adapt.t.size}', 
                             line_color='rgb(221,132,82)', mode='markers+lines', legendgroup='1'), row=1, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=sol_adapt.t, y=err_adapt, name=f'adapted dt, nt={sol_adapt.t.size}', 
                             line_color='rgb(221,132,82)', mode='markers+lines', legendgroup='2'), row=2, col=1)

fig.data[-2].visible = True
fig.data[-1].visible = True

# Create and add slider
steps = []
for i, tol_i in enumerate(tol):
    args = [{"visible": [(el==0) or (el==1) or (el==2) or (el==2*i+3) or (el==2*i+4) for el in range(len(fig.data))]}]
    step = dict(method="update", label = f"{tol_i:.2e}", args=args)
    steps.append(step)
sliders = [dict(active=2, currentvalue={'prefix': 'Tolerance = '}, steps=steps)]
    
legend=dict(x=0.70,y=0.95, bgcolor='rgba(0,0,0,0)')
fig.update_layout(sliders=sliders, height=800, legend=legend, legend_tracegroupgap=270, legend_groupclick="toggleitem")

### Implicit Euler

In [ ]:
Tr = 200.0
beta = 10. 

emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
fcn = emwc1eq.fcn
jac = emwc1eq.jac

tini = 0.0
tend = 5.0

yini = (0.,)

fig =  make_subplots(rows=2, cols=1, vertical_spacing=0.1, subplot_titles=("Solution", "Error"))

sol_ref = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)

nt = 10001
sol = backward_euler(tini, tend, nt, yini, fcn)
sol_tmp = solve_ivp(fcn, (tini, tend), yini, method='Radau', t_eval=sol.t, atol=1.e-8, rtol=1.e-8)
err = np.abs(sol.y[0] - sol_tmp.y[0])

fig.add_trace(go.Scattergl(x=sol_ref.t, y=sol_ref.y[0], name=f'ref. sol.', 
                           line_color='rgb(85,168,104)', mode='markers+lines', legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scattergl(x=sol.t, y=sol.y[0], name=f'fixed dt, nt={sol.t.size}', 
                         line_color='rgb(76,114,176)', mode='markers+lines', legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scattergl(x=sol.t, y=err, name=f'fixed dt, nt={sol.t.size}', 
                         line_color='rgb(76,114,176)', mode='markers+lines', legendgroup='2'), row=2, col=1)


tol = np.array([1e-4, 1e-5, 1e-6])
for tol_i in tol:
    dtini = 1e-4
    sol_adapt = backward_euler_adapted(tini, tend, dtini, yini, fcn, tol=tol_i)
    sol_tmp = solve_ivp(fcn, (tini, tend), yini, method='Radau', t_eval=sol_adapt.t, atol=1.e-8, rtol=1.e-8)
    err_adapt = err = np.abs(sol_adapt.y[0] - sol_tmp.y[0])

    fig.add_trace(go.Scattergl(visible=False, x=sol_adapt.t, y=sol_adapt.y[0], name=f'adapted dt, nt={sol_adapt.t.size}', 
                             line_color='rgb(221,132,82)', mode='markers+lines', legendgroup='1'), row=1, col=1)
    fig.add_trace(go.Scattergl(visible=False, x=sol_adapt.t, y=err_adapt, name=f'adapted dt, nt={sol_adapt.t.size}', 
                             line_color='rgb(221,132,82)', mode='markers+lines', legendgroup='2'), row=2, col=1)

fig.data[-2].visible = True
fig.data[-1].visible = True

# Create and add slider
steps = []
for i, tol_i in enumerate(tol):
    args = [{"visible": [(el==0) or (el==1) or (el==2) or (el==2*i+3) or (el==2*i+4) for el in range(len(fig.data))]}]
    step = dict(method="update", label = f"{tol_i:.2e}", args=args)
    steps.append(step)
sliders = [dict(active=2, currentvalue={'prefix': 'Tolerance = '}, steps=steps)]
    
legend=dict(x=0.70,y=0.95, bgcolor='rgba(0,0,0,0)')
fig.update_layout(sliders=sliders, height=800, legend=legend, legend_tracegroupgap=270, legend_groupclick="toggleitem")